In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from numpy import linalg
from math import *

#Particle in Field Walkthrough


PiF.py is a python file containing classes, methods, and functions pertaining to the creation of simulations of the movement of charged particles in magnetic fields. In this Colab file we will be working through each part of the PiF.py file that has been provided to you.

Because each program will be written out in full here we do not need to upload the .py file to the workspace; simialrly if you would rather copy and paste the contents of PiF.py to your project notebook instead of uploading the file that will work the same.

## Particle and Field Classes

The first part of PiF.py is the classes, of which there are three: particle, b_field, e_field. These are used to build the conditions of our simuations.

### The Particle

The particle class is fairly simple, containing the mass and charge of a particle as well as an optional name which can be used in the later plotting functions.

In [ ]:
class particle:
    def __init__(self, q = 1, m = 1, name = "particle"):
        self.name = name
        self.charge = q
        self.mass = m

Looking at the ```__init__()``` function you can see that each of these attributes are assigned a default value, this means that if we do not have a particular particle in mind just by writing ```particle = particle()``` we will get a default particle object with mass 1 and charge 1, named 'particle'. For now these units are massless but in future projects we can make use of PlasmaPy's particle objects which make use of AstroPy's unit system.

### The b_field

Moving onto our magnetic field object, ```b_field()```. This class defines the magnetic field which is used in our simulations. It has three field related attributes, a graph attribute, and a mathmatical method.

The three field attributes define various parameters used in calculating the magnetic field: the type of field, the magnitude of the field, and the magnetic coefficient, and the number of field lines. The type of field can be one of several options which will calculate the field vector. The magnitude is the magnitude of the field vector, and the magnetic coefficient is _____. The number of field lines is specfically used for drawing the field and particle movement in that field and defines how many field lines to include in the display.

The b_field also has a method called get_field. This uses the magnitude and field type as well as a particle position vector (this needs to be passed into the funtion as it is not contained within the class) to calulate the magnetic field vector at that position.

*It is important to note the data types used. The position vector has to be a (3,0) numpy array and the field vector output is also a (3,0) numpy array. While in this method it does not cause any issues, differently sized data such as (3,1) numpy array or list will cause errors during the integration later in the program.*

In [ ]:
class b_field:
    '''
    This class defines a magnetic field.
    mag is the magnitude of the field.
    btype specifies how the magnetic feild vector should be calcualted from the magnitude:
    "Bx, By, Bz, null, gradB, dipole, bottle, user"
    beta is the coefficient for the magnetic field default = 0
    N is the number of magnetic field lines to show default = 7
    '''
    rRef = 1
    def __init__(self, btype = "null", mag = 1, beta = 0, N = 7):
        self.btype = btype
        self.magnitude = mag
        self.beta = beta
        self.field_lines = N
    def get_field(self, X):
        x = float(X[0])
        y = float(X[1])
        z = float(X[2])
        r = sqrt(x**2 + y**2 + z**2)
        btype = self.btype
        if r != 0:
            match btype:
                case "null":
                    Bx = 0
                    By = 0
                    Bz = 0
                case "Bx":
                    Bx = self.magnitude
                    By = 0
                    Bz = 0
                case "By":
                    Bx = 0
                    By = self.magnitude
                    Bz = 0
                case "Bz":
                    Bx = 0
                    By = 0
                    Bz = self.magnitude
                case "user":
                    Bx = 0
                    By = 0
                    Bz = 0
                case "dipole":
                    Bx = self.magnitude*self.rRef**3*(3*x*z)/r**5
                    By = self.magnitude*self.rRef**3*(3*y*z)/r**5
                    Bz = self.magnitude*self.rRef**3*(3*z**2-r**2)/r**5
                case "bottle":
                    Bx = self.magnitude*(-1*self.beta**2*x*z)
                    By = self.magnitude*(-1*self.beta**2*y*z)
                    Bz = self.magnitude*(1+self.beta**2*z**2)
                case "gradB":
                    Bx = 0
                    By = 0
                    Bz = self.magnitude*(1+self.beta*y)
        else:
            Bx = 0
            By = 0
            Bz = 0
        B = np.array([Bx,By,Bz])
        return B



Let's look at the e_field class next; it is mostly the same as the b_field class. The only difference in the input arguments is that there is no field line parameter which is because we do not need to show the electric field lines. Just like the b_field, e_field has a method called get_field which takes a (3,0) numpy array position and calculated the electric field vector at that position.

In [ ]:
class e_field:
    def __init__(self, etype = "null", mag = 0.01, alpha = 0.05):
        self.etype = etype
        self.magnitude = mag
        self.alpha = alpha
    def get_field(self,X):
        x = float(X[0])
        y = float(X[1])
        z = float(X[2])
        r = sqrt(x**2 + y**2 + z**2)
        etype = self.etype
        if r != 0:
            match etype:
                case "null":
                    Ex = 0
                    Ey = 0
                    Ez = 0
                case "Ex":
                    Ex = self.magnitude
                    Ey = 0
                    Ez = 0
                case "Ey":
                    Ex = 0
                    Ey = self.magnitude
                    Ez = 0
                case "Ez":
                    Ex = 0
                    Ey = 0
                    Ez = self.magnitude
                case "user":
                    Ex = self.magnitude*cos(self.alpha*x)
                    Ey = 0
                    Ez = 0
        else:
            Ex = 0
            Ey = 0
            Ez = 0
        E = np.array([Ex,Ey,Ez])
        return E


## Functions

Now that we have seen the classes, lets look at the functions available to us.

### ODE Solver


The first function is the primary functionality of the program but can be a little confusing due to the nested functions.

The ```solver()``` function uses a set of initial conditions, and a timespan to solve a differential equation to find the position of a particle at numerous different locations through that time period. Let's break it down.

In [ ]:
def solver(particle, efield, bfield, R0, V0, t0, tf):
    def eom(t,X):
        x = X[0]
        y = X[1]
        z = X[2]
        vx = X[3]
        vy = X[4]
        vz = X[5]
        R = np.array([x,y,z])
        V = np.array([vx,vy,vz])
        F = Lorentz_Force(X)
        Fx = F[0]
        Fy = F[1]
        Fz = F[2]
        ax = Fx/m
        ay = Fy/m
        az = Fz/m
        dX = np.array([vx,vy,vz,ax,ay,az])
        return dX

    def Lorentz_Force(X):
        x = X[0]
        y = X[1]
        z = X[2]
        vx = X[3]
        vy = X[4]
        vz = X[5]
        R = np.array([x,y,z])
        E = efield.get_field(R)
        B = bfield.get_field(R)
        V = np.array([vx,vy,vz])
        F = q*(E + np.cross(V,B))
        return F

    def velocity(V,B,vtype = "vpp"):
        b = linalg.norm(B)
        vpp = linalg.norm(np.cross(V,B))/b
        vpr = int(np.dot(V,B))/b
        match vtype:
            case "vpp":
                return vpp
            case "vpr":
                return vpr

    m = particle.mass
    q = particle.charge
    X0 = np.concat([np.transpose(R0),np.transpose(V0)])
    E0 = efield.get_field(X0)
    B0 = bfield.get_field(X0)
    b0 = linalg.norm(B0)
    wg = abs(particle.charge)*b0/particle.mass
    Tg = 2*pi/wg
    vpp0 = velocity(V0,B0, vtype = "vpp")
    rg = vpp0/wg
    N = round(tf/Tg)
    print(f"Simulation Running for {N} Gyro Period")
    t_span = (t0,tf)

    sol = solve_ivp(eom, t_span, X0, max_step = 0.01)
    return sol

Within the solver function we first have to define a few other functions that the solver needs to work. These functions could be defined outside of ```solver``` but by defining them inside we can utilize local variables.

The solver function uses the Runge-Kutta method; in brief this method takes a set of solutions to our function at the start time (initial conditions) and an expression for the derivative of our function. Our first nested function defines this expression of the derivative and is called ```eom()`` for equation of motion.

The ```eom()``` function needs a (6,0) input defining the starting position and velocity vectors (x,y,z,vx,vy,vz). Assuming our particle is moving fairly slow we can then use $\vec{F} =m\vec{a}$ to find our acceleration. In this case the particle will be impacted by the Lorentz Force which is defined by $\vec{F} = q(\vec{v}\times\vec{B} + \vec{E})$. The ```eom()``` function makes use of another function also defined within ```solver()``` to calculate this force. This function is called ```Lorentz_Force()```.

```
    def Lorentz_Force(X):
        x = X[0]
        y = X[1]
        z = X[2]
        vx = X[3]
        vy = X[4]
        vz = X[5]
        R = np.array([x,y,z])
        E = efield.get_field(R)
        B = bfield.get_field(R)
        V = np.array([vx,vy,vz])
        F = q*(E + np.cross(V,B))
        return F
```

Because the ```Lorentz_Force()``` function is defined within our ```solver()``` function, we do not need to pass e_field and b_field objects into it as parameters because it already has access to them as local variables. Lorentz_Force uses the same position and velocity values as ```eom()```, calculates the electric and magnetic field vectors using ```.get_field()```, and then calculates the Lorentz Force at that location. This result is returned and can be used by ```eom()``` to find the acceleration at that point.



#### Variable Locations
Before we move on here are some examples showing global and local variables in more depth.

In [12]:
global_variable = "hello from global" #this variable is defined outside of a function and therefore can be accessed
def outer(passed_parameter):
  local_variable = "hello from outer"
  print(local_variable)
  print(global_variable)
  print(passed_parameter)

  def inner():
    local_variable = "hello from inner" #this variable is defined inside a function and therefore cannot be accessed outside of the function
    print(local_variable)
    print(global_variable)
    print(passed_parameter)
  print("running inner()")
  inner()



outer("this is what you passed into outer()")

hello from outer
hello from global
this is what you passed into outer()
running inner()
hello from inner
hello from global
this is what you passed into outer()


The global variable is available no matter what, at all locations in the code. But notive that even though local_variable has the same name it has a different value depending on which part of the code you are running. Notice also that the parameter you passed into the outer function remains the same no matter what.

This is the same principal we used in the nested functions for the ```solver()``` function. the b_field, e_field, and particle attributes are not global variables so they cannot be accessed by the functions within ```solver()``` unless they are provided, by nesting ```Lorentz_Force()``` inside of solver we only need to pass our fields in once and they can then be used by all inner functions.

#### Back to ```Solver()```

Before we go over the solving part of ```solver()``` there is one last nested function:
```
def velocity(V,B,vtype = "vpp"):
    b = linalg.norm(B)
    vpp = linalg.norm(np.cross(V,B))/b
    vpr = int(np.dot(V,B))/b
    match vtype:
        case "vpp":
            return vpp
        case "vpr":
            return vpr
```
This function determines the magnitude of the particle's velocity parallel and perpendicular to the magnetic field. In most cases, the perpendicular velocity is all we need as the perpendicular velocity is what is needed to find the radius of gyration, gyration frequency, and period.

The magnetic force on a particle moving within it will cause the particle to move in a circular motion, the radius of this motion can be calulated by assuming the Lorentz Force is the centrepital force. Equating these equations results in

$-\frac{mv^2}{r}\vec{r} = q\vec{v}\times\vec{B}$

which when the velocity is perpendicular simplifies to

$\frac{mv^2}{r} = qvB$

The radius of gyration (cyclotron radius) can be found by solving for r, which can then be used to find the angular frequency of gyration which is used to find the period.

$r_{g,c} = \frac{mv}{qB}$

$\omega_g = 2\pi\nu = 2\pi(\frac{v}{2\pi r}) = \frac{qB}{m}$

$T_g = \frac{2\pi}{\omega_g}$

This is all done by the lines of code in ```solver()``` after the velocity function.

```
m = particle.mass
q = particle.charge
X0 = np.concat([np.transpose(R0),np.transpose(V0)])
E0 = efield.get_field(X0)
B0 = bfield.get_field(X0)
b0 = linalg.norm(B0)
wg = abs(particle.charge)*b0/particle.mass
Tg = 2*pi/wg
vpp0 = velocity(V0,B0, vtype = "vpp")
rg = vpp0/wg
N = round(tf/Tg)
print(f"Simulation Running for {N} Gyro Period")
t_span = (t0,tf)
```

Once the frequency is found it can be used with the final time to find how many gyrations have been done. This is done before the simulation is ran and printed to the terminal.

**```solve_ivp()```**


The real heavy lifting of the ```solver()``` function is the ```solve_ivp()``` function that we imported from scipy, a Python package for scientific and technical computing. the solve_ivp function can utilize a number of numerical integration methods but by default, and in this case, uses the 5th order  Runge-Kutta method.

Here is some of the official documentation for the ```solve_ivp()``` function from Scipy.

```solve_ivp(fun, t_span, y0, method='RK45', t_eval=None, dense_output=False, events=None, vectorized=False, args=None, **options)```

**```Parameters:```**

**fun : callable**

> Right-hand side of the system: the time derivative of the state y at time t. The calling signature is fun(t, y), where t is a scalar and y is an ndarray with len(y) = len(y0). Additional arguments need to be passed if args is used (see documentation of args argument). fun must return an array of the same shape as y. See vectorized for more information.

**t_span : 2-member sequence**

> Interval of integration (t0, tf). The solver starts with t=t0 and integrates until it reaches t=tf. Both t0 and tf must be floats or values interpretable by the float conversion function.

**y0 : array_like, shape (n,)**

> Initial state. For problems in the complex domain, pass y0 with a complex data type (even if the initial value is purely real).

**max_step : float, _optional_**

> Maximum allowed step size. Default is np.inf, i.e., the step size is not bounded and determined solely by the solver.



  



Our ```eom()``` function is passed as the function for ```solve_ivp()```, it takes a y0 value as a (6,) nparray with the form (x,y,z,vx,vy,vz) and outputs a (6,) nparray with the form (vx,vy,vz,ax,ay,az) as the time derivative of the input.

t_span is easy enough as it is just a list or tuple containing our t0 and tf values.

y0 is also easy since it is just the (6,) nparray containing the initial position and velocities.

Finally we use the optional ```max_step``` parameter to force the solver to perform a calculation every 0.1 seconds. If you remove this parameter or make it bigger you will see the path become less smooth as the number of points calculated becomes smaller.


*How does RK45 work*

##Simulation


Once the simulation is ran and all points are calculated we can plot these points in space to visualize the movement of the particle. All of this is done within the ```simulate()``` function.

```
def simulate(initial_conditions, speed = 'real', title = 'Simulation of Particle in a Field', view = "3D"):

    particle, efield, bfield, R0, V0, t0, tf = initial_conditions
    sol = solver(particle, efield, bfield, R0, V0, t0, tf)
    t = sol.t
    x = sol.y[0]
    y = sol.y[1]
    z = sol.y[2]

    fig = plt.figure()
    match view:
        case "3D":
            ax = fig.add_subplot(111, projection='3d')
            ax.plot(x, y, z, color='black', label='particle path')
            ax.set_xlabel('x(m)')
            ax.set_ylabel('y(m)')
            ax.set_zlabel('z(m)')
            ax.view_init(elev=20, azim=45)
            particle_point, = ax.plot([], [], [], 'ro', label = particle.name)
            time_text = ax.text2D(0.05, 0.95, '', transform=ax.transAxes)
        case "x-y":
            ax = fig.add_subplot(111)
            ax.plot(x,y,color='black',label = 'particle path')
            ax.set_xlabel('x(m)')
            ax.set_ylabel('y(m)')
            particle_point, = ax.plot([], [], 'ro', label = particle.name)
            time_text = ax.text(0.05, 0.95, '', transform=ax.transAxes)
        case "y-z":
            ax = fig.add_subplot(111)
            ax.plot(y,z,color='black',label = 'particle path')
            ax.set_xlabel('y(m)')
            ax.set_ylabel('z(m)')
            particle_point, = ax.plot([], [], 'ro', label = particle.name)
            time_text = ax.text(0.05, 0.95, '', transform=ax.transAxes)
        case "x-z":
            ax = fig.add_subplot(111)
            ax.plot(x,z,color='black',label = 'particle path')
            ax.set_xlabel('x(m)')
            ax.set_ylabel('z(m)')
            particle_point, = ax.plot([], [], 'ro', label = particle.name)
            time_text = ax.text(0.05, 0.95, '', transform=ax.transAxes)

    ax.grid(False)
    ax.set_title(title)
    ax.legend()



    def update(num, x, y, z, particle_point, time_text):
        if view == "3D":
            particle_point.set_data([x[num]], [y[num]])  # Pass sequences
            particle_point.set_3d_properties([z[num]])  # Pass sequences
        elif view == "x-y":
            particle_point.set_data([x[num]], [y[num]])  # Pass sequences
        elif view == "y-z":
            particle_point.set_data([y[num]], [z[num]])  # Pass sequences
        elif view == "x-z":
            particle_point.set_data([x[num]], [z[num]])  # Pass sequences
        time_text.set_text(f'Time: {t[num]:.2f}s')
        return particle_point, time_text

    #blit = False for Jupyter Lab, blit = true otherwise
    match speed:
        case 'real':
            interval = 0
        case 'slow':
            interval = 2
    ani = FuncAnimation(fig, update, frames=len(t), fargs=(x, y, z, particle_point, time_text), interval = interval, blit=False, repeat=False)

    global animation
    animation = ani
    
    plt.show()
    return ani
  ```

  This is a very large function, so we will start by just looking at the three main things it does: calculate the trajectory, plot the trajectory as a still image, animate a particle moving along that trajectory in time.

  The first task has pretty much already been done, we just need to pass in the initial conditions and ```sovler()``` does the rest.
  
  The second task is contained within the match-case statement. Each case is a different view that you may want: if you want a 2D view it can be either x-y,x-z,y-z and each case then uses ```ax.plot(x,y)``` to plot the desired trajectory; or 3D uses ```ax.plot3D(x,y,z)``` to generate a 3D plot. These case statements also create a point representing the particle, some axis labels, and a text label which dispays the time in seconds. With only the output from this match-case statement we would have a plot with teh trajectory, a stationary point, and a label that says 0.00 seconds, which is not very useful.

  To animate the plot we use the update function which we defined. This function takes the entire x, y, and z (if applicable) arrays and changes the point location to a point in those arrays. It can also update the time label so that it shows a different time each update. By combining this function with matplotlib's ```FuncAnimation()``` Python will plot the point and then update the points location over and over again, moving through the trajectory one point at a time for the duration of the trajectory. This creates an animation which can then be saved as a .gif.
  
  
  To see all of this in action move onto the next section and begin working through the Particle in Field Project.